In [1]:
using VCFTool

┌ Info: Precompiling VCFTool [8f8d80d9-e796-45a3-8ef2-c0461a3110a4]
└ @ Base loading.jl:1278


In [2]:
# For viewing DataFrames

ENV["COLUMNS"]=100

ENV["LINES"]=200

# For viewing JuliaDB tables and arrays

using IndexedTables

IndexedTables.set_show_compact!(false)

false

In [4]:
input_dir = "../input/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_738_1000_file_path = joinpath(input_dir, "738_variants_1000.vcf.gz")

vcf_769_3_file_path = joinpath(input_dir, "769_3_variants.vcf.gz")

vcf_test_file_path = joinpath(input_dir, "test.vcf.gz")

vcf_file_path_to_use = vcf_test_file_path

"../input/test.vcf.gz"

In [ ]:
# timerinfo, tmp = tracktime() do
#          println("yes")
# end

## Search by Variant

In [ ]:
vcf = make_vcf_ndsparse(vcf_file_path_to_use)

In [ ]:
vcf_table = make_vcf_indexedtable(vcf_file_path_to_use)

In [ ]:
# https://www.snpedia.com/index.php/Rs1042522
rs1042522 = ["chr17", 7676154]

# TP53
tp53 = ["chr17", 7661779:7687550]

# test set
t = ["chr1", 10000:12000]

## Get Genotype

In [ ]:
using Dates

start_time = now()

for value in vcf["chr17", 7661779]
    
    genotype = []
    
    allele_ref = value[:REF]
    
    allele_alt = value[:ALT]
    
    numeric_genotype = split(split(value[:GERM], ":")[1], "/")
    
    if occursin("|", numeric_genotype[1])
        
        numeric_genotype = split(numeric_genotype[1], "|")
        
    end
    
    counter = 1
    
        while counter < 3
        
            if "0" == numeric_genotype[counter]

                push!(genotype, allele_ref)

            elseif "1" == numeric_genotype[counter]

                push!(genotype, split(allele_alt, ",")[1])

            elseif "2" == numeric_genotype[counter]

                push!(genotype, split(allele_alt, ",")[2])
        
            end    
    
        counter += 1
        
        end

    println("Genotype: $genotype")
    
    println("\n")
    
end
  
end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")

In [ ]:
import Pkg; Pkg.add("Dagger")

using Dagger

Dagger.save(vcf, "../input/test_vcf_ndsparse_save.vcf")

In [ ]:
using Dagger

dagger_load_vcf = Dagger.load("../input/test_vcf_ndsparse_save.vcf")

## Search by Gene

## Search by Region

## Tabix Regions

In [ ]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)

## JuliaDB

An unzipped VCF file can be loaded into an indextable or ndsparse array directly using loadtable() or ndsparese() respectively. However, its much faster to query variants if they are in an ndsparse array because the position and position are both part of the index.